In [1]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as F

import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary, \
    vectorize_texts, SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

In [46]:
corpus = ['Казнить нельзя помиловать Нельзя наказывать','Казнить нельзя помиловать Нельзя освободить', 
          'Нельзя не помиловать', 'Обязательно освободить']

In [47]:
clean = lambda x: x.lower().split()
corpus_tokens = list(map(clean, corpus))

In [48]:
corpus_tokens

[['казнить', 'нельзя', 'помиловать', 'нельзя', 'наказывать'],
 ['казнить', 'нельзя', 'помиловать', 'нельзя', 'освободить'],
 ['нельзя', 'не', 'помиловать'],
 ['обязательно', 'освободить']]

In [49]:
vocabulary, word_doc_freq = build_vocabulary(corpus_tokens, min_count=0)

In [50]:
vocab = dict(zip(vocabulary.keys(), list(word_doc_freq)))

In [52]:
vocab = {k: v for k, v in sorted(vocab.items(), key=lambda item: item[1])}

In [71]:
vocab_nums = dict(zip(vocab.keys(),[0,1,2,3,4,5,6]))

In [76]:
vectorize_texts(corpus_tokens, vocab_nums, vocab.values(), mode='tf', scale=False).toarray()

array([[0.2       , 0.        , 0.        , 0.2       , 0.        ,
        0.4       , 0.2       ],
       [0.        , 0.        , 0.        , 0.2       , 0.2       ,
        0.4       , 0.2       ],
       [0.        , 0.33333334, 0.        , 0.        , 0.        ,
        0.33333334, 0.33333334],
       [0.        , 0.        , 0.5       , 0.        , 0.5       ,
        0.        , 0.        ]], dtype=float32)

In [74]:
vectorize_texts(corpus_tokens, vocab_nums, vocab.values(), mode='idf', scale=False).toarray()

TypeError: unsupported operand type(s) for /: 'int' and 'dict_values'

In [160]:
import scipy.sparse
def tf_idf(tokenized_texts, word2id, word2freq):
    
    result = scipy.sparse.dok_matrix((len(tokenized_texts), len(word2id)), dtype='float32')
    for text_i, text in enumerate(tokenized_texts):
        for token in text:
            if token in word2id:
                result[text_i, word2id[token]] += 1
                
     # получаем вектора относительных частот слова в документе для tf
    result_tf = result.tocsr()
    result_tf = result_tf.multiply(1 / result.sum(1)).toarray()

    # полностью убираем информацию о количестве употреблений слова в данном документе,
    # но оставляем информацию о частотности слова в корпусе в целом для idf
    result_idf = result.tocsr()
    result_idf = (result_idf>0).astype('float32').multiply(1 / np.array(list(word2freq))).toarray()
    
    matrix = np.log1p(result_tf)*result_idf
    std = np.std(matrix, axis=0, ddof=1)
    mean = np.mean(matrix, axis=0)
    
    for i in range(matrix.shape[1]):
        matrix[:,i] = (matrix[:,i] - mean[i])/std[i]
    
    print(np.std(matrix, axis=0, ddof=1))
    print(np.mean(matrix, axis=0))
    
    return matrix

In [161]:
tf_idf(corpus_tokens, vocab_nums, list(vocab.values()))

[1.        1.        1.        1.        1.        0.9999999 0.9999999]
[ 0.0000000e+00  0.0000000e+00 -5.9604645e-08  0.0000000e+00
  0.0000000e+00  5.9604645e-08  0.0000000e+00]


array([[ 1.5       , -0.5       , -0.50000006,  0.8660254 , -0.75312984,
         0.809803  ,  0.1428922 ],
       [-0.5       , -0.5       , -0.50000006,  0.8660254 ,  0.14848864,
         0.809803  ,  0.1428922 ],
       [-0.5       ,  1.5       , -0.50000006, -0.8660254 , -0.75312984,
        -0.37570813,  1.0650662 ],
       [-0.5       , -0.5       ,  1.5       , -0.8660254 ,  1.357771  ,
        -1.2438977 , -1.3508506 ]], dtype=float32)

In [125]:
vocab

{'наказывать': 0.25,
 'не': 0.25,
 'обязательно': 0.25,
 'казнить': 0.5,
 'освободить': 0.5,
 'нельзя': 0.75,
 'помиловать': 0.75}